In [1]:
%cd "C:\Users\MANGAL\Desktop\Docking\dockmate"
%pip install . 

C:\Users\MANGAL\Desktop\Docking\dockmate
^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dockmate import Protein, Ligand, PocketFinder, VinaDocking
from dockmate.utils import view_molecule, clean_temp_folder, fetch_pdb, fetch_sdf
 
clean_temp_folder()

✅ Temp folder cleaned: C:\Users\MANGAL\Desktop\Docking\dockmate\temp


In [2]:
fetch_pdb(pdb_id = "1HVR")

WindowsPath('C:/Users/MANGAL/Desktop/Docking/1HVR.pdb')

In [3]:
protein = Protein('1HVR.pdb')
protein.prepare()
# protein.save_pdbqt()
# view_molecule(file_path = protein.pdbqt_path, style = "cartoon")


In [4]:
pf = PocketFinder(protein.pdb_path)
pockets = pf.run()
# pf.save_report()
pockets

[{'rank': 1, 'center': (-9.2421, 16.1006, 27.6026)},
 {'rank': 2, 'center': (-12.5447, 21.7728, 45.5697)}]

In [5]:
fetch_sdf(cid = "338")

WindowsPath('C:/Users/MANGAL/Desktop/Docking/338.sdf')

In [6]:
ligand = Ligand("338.sdf")
ligand.prepare(minimize=None)
# ligand.save_pdbqt()
# view_molecule(file_path = ligand.pdbqt_path)

#### Perform protein ligand docking at the first pocket

In [7]:
#Perform docking with the first pocket
vina = VinaDocking(
    receptor=protein.pdbqt_path,
    ligand=ligand.pdbqt_path,
    center=pockets[0]['center'], 
    size=(40, 40, 40),
    cpu=4,
)

vina.run()
vina.save_results(f"vina_results/{protein.pdbqt_path.stem}_{ligand.pdbqt_path.stem}_pocket_1_docking")



#### Perform protein ligand docking at all the predicted protein pockets


In [8]:
#Loop through each pocket and perform docking
for pocket in pockets:
    center = pocket["center"]
    rank = pocket["rank"]

    vina = VinaDocking(
        receptor=protein.pdbqt_path,
        ligand=ligand.pdbqt_path,
        center=center,
        size=(40, 40, 40),
        exhaustiveness=8,
        num_modes=9
    )
    vina.run()
    vina.save_results(f"vina_results/{protein.pdbqt_path.stem}_{ligand.pdbqt_path.stem}_pocket_{rank}_docking")
